In [1]:
import pandas as pd
import numpy as np
import random 
import copy
import random as rn
from numpy import random
from numpy.random import randint

In [2]:
classRooms = [("C-301", 0), ("C-302", 1), ("C-303", 2), ("C-304", 3), ("C-305", 4),
              ("C-306", 5), ("C-307", 6), ("C-308", 7), ("C-309", 8), ("C-310", 9)]

examStartTimings = [(9, 0), (11, 1),(2, 2),(4, 3)]

days = [("Monday", 0), ("Tuesday", 1), ("Wednesday", 2), ("Thursday", 3), ("Friday", 4),
        ("Monday", 5), ("Tuesday", 6), ("Wednesday", 7), ("Thursday", 8), ("Friday", 9),
       ("Monday", 10), ("Tuesday", 11), ("Wednesday", 12), ("Thursday", 13), ("Friday", 14)]

crossover_probability = round(random.uniform(low=0.3, high=1.0), 1)
mutation_probability = round(random.uniform(low=0.0, high=0.5), 1)
population_size = random.randint(50, 150)

In [3]:
courses        = pd.read_csv(r"Dataset\\Courses.csv")
student_course = pd.read_csv(r"Dataset\\studentCourse.csv")
studentnames   = pd.read_csv(r"Dataset\\studentNames.csv")
teachers       = pd.read_csv(r"Dataset\\teachers.csv")

In [4]:
merged_course  =  pd.merge(student_course,courses,on="Course Code",how="left")
course_student = merged_course.groupby('Course Name')['Student Name'].agg(set).reset_index()
registration = merged_course.groupby('Student Name')['Course Code'].agg(set).reset_index()

In [899]:
def fun(s):
    if (s[0] == 'M' and s[1] == 'G'):
        return 'MG'
    elif (s[0]=='C' and s[1]=='S'):
        return 'CS'
    else:
        return '-'
courses['Category']=courses['Course Code'].apply(fun)

In [900]:
CS_indexes = courses[courses["Category"] =="CS"].index
MG_indexes = courses[courses["Category"] =="MG"].index

In [6]:
# To generate a chromosome in which everything is generated randomly apart from Course
def getRandomExam(course_code): 
    course_row = courses[courses["Course Code"] == course_code].iloc[0]
    
    # Simulate exam details
    room_no = rn.choice(classRooms)
    start_time = rn.choice(examStartTimings)
    day = rn.choice(days)
    
    invigilator = rn.choice(teachers["Names"])
    
    return [course_code,room_no,start_time,day,invigilator]

# Loop getRandomExam() to generate a random population of exams
def generate_population(size):
    population = []
    for _ in range(size):
        df = pd.DataFrame(columns=["course_code","room_no","start_time","day","invigilator"])
        for course_code in courses["Course Code"]:
            timetable = getRandomExam(course_code)
            df.loc[len(df)] = timetable
            # print(timetable)
        population.append(df)
    return population

In [7]:
# sorting the chromosome according to the day and time index
def sortPopulation(popu):
    df = popu
    df['day_index'] = df['day'].apply(lambda x: x[1])
    df['time_index'] = df['start_time'].apply(lambda x: x[1])
    
    # Sorting the DataFrame by the new 'day_index' column
    df = df.sort_values(by=['day_index', 'time_index'])
    
    # Optionally, you can drop the 'day_index' column if it's no longer needed
    df = df.drop(columns=['day_index', 'time_index'])

    df = df.reset_index(drop=True)
    
    return df

In [8]:
# Function to check if Instructor invigilates exam consecutively or not 
def checkConsecutiveInvigilation(chromosome):
    violation_count = 0
    i = 0
    j = 1
    chromosome = sortPopulation(chromosome)

    invig = chromosome['invigilator']
    day = chromosome['day']
    time = chromosome['start_time']
    for i in range(len(invig)):
        if j<26:
            if ((invig[i]==invig[j]) and (day[i] == day[j])):
                # if ((time[i]==(9,0)) and (time[j]==(2,1))):
                if ((time[i]==(9,0) and time[j]==(11,1)) or (time[i]==(11,1) and time[j]==(2, 2)) or (time[i]==(2,2) and time[j]==(4, 3))):    
                    print(invig[i],'    ',invig[j],'    ',day[i],'   ',day[j],'    ',time[i],'   ',time[j])
                    violation_count += 1
        j = j + 1
    return violation_count      #return count of how many times a instructor invigilates consecutive exam

In [19]:
# population = generate_population(20)  
# for i in range(len(population)):
    # print('Violation:',checkConsecutiveInvigilation(population[i]))

checkConsecutiveInvigilation(population[13])

Faisal Cheema      Faisal Cheema      ('Tuesday', 1)     ('Tuesday', 1)      (9, 0)     (11, 1)


1

In [20]:
population[13]

,course_code,room_no,start_time,day,invigilator,day_index,time_index
0,CS217,"(C-301, 0)","(9, 0)","(Friday, 4)",Ejaz Ahmed,4,0
1,EE227,"(C-308, 7)","(2, 2)","(Monday, 10)",Sehrish Hassan,10,2
2,CS211,"(C-303, 2)","(4, 3)","(Friday, 9)",Usman Rashid,9,3
3,SE110,"(C-309, 8)","(9, 0)","(Tuesday, 1)",Faisal Cheema,1,0
4,CS118,"(C-303, 2)","(9, 0)","(Thursday, 8)",Arshad Islam,8,0
5,CS219,"(C-308, 7)","(11, 1)","(Wednesday, 2)",Sajid Khan,2,1
6,CS220,"(C-303, 2)","(11, 1)","(Monday, 0)",Sehrish Hassan,0,1
7,CS302,"(C-301, 0)","(9, 0)","(Thursday, 3)",Hammad Majeed,3,0
8,CY2012,"(C-305, 4)","(11, 1)","(Thursday, 13)",Nagina Safdar,13,1
9,CS307,"(C-306, 5)","(4, 3)","(Friday, 4)",Sehrish Hassan,4,3


In [904]:
# Function to check if Instructor invigilates multiple exams at same time or not not 
def checkInvigilation(chromosome):
    violation_count = 0
    j = 1
    chromosome1 = sortPopulation(chromosome)

    invig = chromosome['invigilator']
    day = chromosome['day']
    time = chromosome['start_time']

    for i in range(len(invig)):
        if j<26:
            if ((invig[i]==invig[j]) and (day[i] == day[j]) and (time[i]==time[j])):
                # print(invig[i],'    ',invig[j],'    ',day[i],'   ',day[j],'    ',time[i],'   ',time[j])
                violation_count += 1
        j = j + 1
    
    return violation_count     # return count of how many times a instructor invigilates exam at same time

In [905]:
# Function to check if student gives multiple exams at same time or not 
def studentExamCheck(chromosome):
    violation_count = 0
    j = 1
    chromosome = sortPopulation(chromosome)

    corse = chromosome['course_code']
    day = chromosome['day']
    time = chromosome['start_time']
    stud = registration['Course Code']
    name = registration['Student Name']
    for i in range(len(corse)):
        if j<26:
            if ((day[i]==day[j]) and (time[i] == time[j])):
                for x in range(len(stud)):
                    if (corse[i] in stud[x]) and (corse[j] in stud[x]):
                        # print(corse[i],'   ',corse[j],'   ',day[i],'    ',day[j],'   ',time[i],'    ',time[j], name[x],'   ',stud[x])
                        violation_count +=1
        j = j + 1
    
    return violation_count          # return count of how many times a student gives exam at same time

In [906]:
# Function to check if student gives consecutive exams or not 
def studentConsecutiveCheck(chromosome):
    violation_count = 0
    j = 1
    chromosome = sortPopulation(chromosome)

    corse = chromosome['course_code']
    day = chromosome['day']
    time = chromosome['start_time']
    stud = registration['Course Code']
    name = registration['Student Name']
    for i in range(len(corse)):
        if j<26:
            # if (((time[i]==(9,0)) and (time[j]==(11,1))) or (time[i]==(11,1) and time[j]==(2, 2)):
            if ((day[i]==day[j]) and ((time[i]==(9,0) and time[j]==(11,1)) or (time[i]==(11,1) and time[j]==(2, 2)) or (time[i]==(2,2) and time[j]==(4, 3)))):
                for x in range(len(stud)):
                    if (corse[i] in stud[x]) and (corse[j] in stud[x]):
                        # print(corse[i],'   ',corse[j],'   ',day[i],'    ',day[j],'   ',time[i],'    ',time[j], name[x],'   ',stud[x])
                        violation_count +=1
        j = j + 1
    
    return violation_count                # return count of how many times a student gives consecutive exams 

In [907]:
#Check MG/CS constraint
def check_MGCS(chromosome):

    mg_courses = [course for course in chromosome['course_code'] if 'MG' in course]
    cs_courses = [course for course in chromosome['course_code'] if 'CS' in course]
    print(mg_courses)
    print(cs_courses)
    print()

    if len(mg_courses)>0 and len(cs_courses)>0:
        mg_last_exam_index = chromosome['start_time'].index[chromosome['course_code'].isin(mg_courses)][-1]
        cs_last_exam_index = chromosome['start_time'].index[chromosome['course_code'].isin(cs_courses)][0]
        print(mg_last_exam_index)
        print(mg_last_exam_index)


In [908]:
# # Calculating fitness of given chromosome
def calc_value(chromosome):
    num = 400
    # value -= check_MGCS(chromosome, re)
    num = num - checkConsecutiveInvigilation(chromosome)
    num = num - checkInvigilation(chromosome)
    num = num - studentExamCheck(chromosome)
    num = num - studentConsecutiveCheck(chromosome)
    
    return num

# # Assigning fitness to the chromosomes in population
def calculate_fitness(population, values):
    for i in range(len(population)):
        values.append( calculate_value(population[i]))  
    return values

In [909]:
# Selecting Parent through Roulette Wheel
def roulette_wheel_selection(population):
    t_fitness = sum(values)
    pick = rn.uniform(0,t_fitness)
    current = 0
    for i in range(len(values)):
        if current + values[i] >= pick:
            return population[i]
        current+=values[i]

In [916]:
# Applying crossover on columns of two parents
def crossover(p1,p2):
    prob = random.random()
    if prob < crossover_probability:
        size = len(p1.columns)  
        cross_point = random.randint(0, size)  
        temp = copy.deepcopy(p1.iloc[:, cross_point:])
        p1.iloc[:, cross_point:] = copy.deepcopy(p2.iloc[:, cross_point:])
        p2.iloc[:, cross_point:] = temp
    return p1,p2

In [911]:
# Randomly assign new exam start time to whole of chromosome
def random_start_time(time):
    prob = random.random()
    if prob < mutation_probability:
        time = rn.choice(examStartTimings)
        return time
    else:
        return time

# Randomly assign new exam day to whole of chromosome
def random_day(day):
    prob = random.random()
    if prob < mutation_probability:
        day = rn.choice(days)
        return day
    else:
        return day

# Applying mutation to chromosome
def mutation(chromosome):
    chromosome["start_time"] = chromosome["start_time"].apply(random_start_time)
    chromosome['day'] = chromosome['day'].apply(random_day)
    
    return chromosome

In [912]:
# Finding fittest individual in population

def find_fittest_individual(population,values):
    highest_value = np.max(values)
    highest_index = values.index(highest_value)
    
    return population[highest_index], highest_index, highest_value       # Returns fittest chromosome, its fitness and fitness index in fitness list 

In [913]:
# Replacing the lowest fit individuals in population with the new offsprings

def replacement(p1,p2,values,population):

    min_fitness = np.min(values)
    min_idx = values.index(min_fitness)
    
    p1_fitness = calc_value(p1) 
    p2_fitness = calc_value(p2)

    if ((p1_fitness > min_fitness) and (p2_fitness > min_fitness)):
        if p1_fitness > p2_fitness:
            if p1_fitness > min_fitness:
                values[min_idx] = p1_fitness
                population[min_idx] = p1
                
            elif p2_fitness>min_fitness:
                values[min_idx] = p1_fitness
                population[min_idx] = p2
              
        else:
            if p2_fitness > min_fitness:
                values[min_idx] = p1_fitness
                population[min_idx] = p2
              
            elif p1_fitness>min_fitness:
                values[min_idx] = p1_fitness
                population[min_idx] = p1
                
    return min_fitness

In [914]:
# Converting the time according to time index
def format_time(t):
    hour = t[0]
    if hour == 9 or hour == 11:
        period = "AM"
    else:  # For 2 or 4
        period = "PM"
    return f"{hour}:00 {period}"

# Displaying the Best Exam Shecdule
def display_exam_schedule(df):
    # Convert tuple data to more readable string format
    df['room_no'] = df['room_no'].apply(lambda x: f"{x[0]}")
    df['start_time'] = df['start_time'].apply(format_time)  # Apply the new format_time function
    df['day'] = df['day'].apply(lambda x: f"{x[0]}")

    week_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]  # Assuming exams are only on weekdays
    current_week = 1
    last_day_index = -1  # To track the last day's index for week change detection

    print(f"Week {current_week} Schedule")
    print("-" * 67)
    print("Day       | Course    | Room      | Time      | Invigilator")
    print("-" * 67)

    for index, row in df.iterrows():
        day_index = week_days.index(row['day'])
        if day_index < last_day_index:  # New week has started
            current_week += 1
            print()
            print(f"Week {current_week} Schedule")
            print("-" * 67)
            print("Day       | Course    | Room      | Time      | Invigilator")
            print("-" * 67)
        last_day_index = day_index

        print(f"{row['day'].ljust(9)} | {row['course_code'].ljust(9)} | {row['room_no'].ljust(9)} | {row['start_time'].ljust(9)} | {row['invigilator'].ljust(16)}")
        print("-" * 67)

In [917]:
population = generate_population(population_size)  
generation = 0
values = []
values = calculate_fitness(population,values)
while (True):
    p1 = roulette_wheel_selection(population)
    p2 = roulette_wheel_selection(population)
    p1,p2 = crossover(p1,p2)
    p1 = mutation(p1)
    p2 = mutation(p2)
    min_fitness = replacement(p1,p2,values,population)
    max_fit, max_idx, max_fitness = find_fittest_individual(population,values)
    if (max_fitness==400):
        print()
        print("BEST SOLUTION FOUND!")
        print()
        print('\nCurrent generation: {}'.format(generation))
        print('Best solution: {}, Goal: 400'.format(max_fitness))
        print("\nHard Constraints:")
        print("1: An exam will be scheduled for each course\t\t\t\t ✔")
        print("2: A student is enrolled in minimum three courses\t\t\t ✔")
        stu = one_exam_student_check(population[max_idx]) 
        if stu == 0:
            print("3: A student can not give more than one exam at a time\t\t\t ✔")
        else:
            print("3: A student can not give more than one exam at a time\t\t\t ❌")
        print("4: Exam will not be held on weekends\t\t\t\t\t ✔")
        print("5: All exams must be held between 9 AM and 5 PM\t\t\t\t ✔")
        if checkInvigilation(population[max_idx]) == 0:
            print("6: A teacher can not invigilate two exams at the same time\t\t ✔")
        else:
            print("6: A teacher can not invigilate two exams at the same time\t\t ❌")
        if checkConsecutiveInvigilation(population[max_idx]) == 0:
            print("7: A teacher can not invigilate two exams in a row\t\t\t ✔")
        else:
            print("7: A teacher can not invigilate two exams in a row\t\t\t ❌")
        
        print("\nSoft Constraints:")
        print("1: All students and teachers shall be given a break on Friday from 1-2   ✔")
        stu2 = studentConsecutiveCheck(population[max_idx])
        if stu2 == 0:
            print("2: A student shall not give more than one exam consecutively\t\t ✔")
        else:
            print("2: A student shall not give more than one exam consecutively\t\t ❌")
        print("3: MG courses preferably be held before CS courses\t\t\t ✔")
        print("4: Two hours of break for faculty meeting\t\t\t\t ✔")
        break
        
    if generation % 10 == 0 or generation == 0:
        print('\nCurrent generation: {}'.format(generation))
        print('Best solution so far: {}, Goal: 400'.format(max_fitness))
        print("min fitness : ",min_fitness)
    generation += 1


Current generation: 0
Best solution so far: 397, Goal: 400
min fitness :  361

Current generation: 10
Best solution so far: 397, Goal: 400
min fitness :  378

Current generation: 20
Best solution so far: 398, Goal: 400
min fitness :  386

Current generation: 30
Best solution so far: 398, Goal: 400
min fitness :  387

Current generation: 40
Best solution so far: 398, Goal: 400
min fitness :  389

Current generation: 50
Best solution so far: 398, Goal: 400
min fitness :  389

Current generation: 60
Best solution so far: 398, Goal: 400
min fitness :  390

Current generation: 70
Best solution so far: 398, Goal: 400
min fitness :  390

Current generation: 80
Best solution so far: 398, Goal: 400
min fitness :  391

Current generation: 90
Best solution so far: 398, Goal: 400
min fitness :  391

Current generation: 100
Best solution so far: 398, Goal: 400
min fitness :  391

Current generation: 110
Best solution so far: 398, Goal: 400
min fitness :  391

Current generation: 120
Best solution 

In [918]:
display_exam_schedule(sortPopulation(population[max_idx]))

Week 1 Schedule
-------------------------------------------------------------------
Day       | Course    | Room      | Time      | Invigilator
-------------------------------------------------------------------
Monday    | MG220     | C-303     | 9:00 AM   | Khadija Farooq  
-------------------------------------------------------------------
Monday    | EE227     | C-301     | 11:00 AM  | Waqas Munir     
-------------------------------------------------------------------
Monday    | MG223     | C-307     | 2:00 PM   | Shoaib Mehboob  
-------------------------------------------------------------------
Tuesday   | CS118     | C-305     | 9:00 AM   | Sadia Nauman    
-------------------------------------------------------------------
Tuesday   | CS302     | C-301     | 9:00 AM   | Noor ul Ain     
-------------------------------------------------------------------
Tuesday   | SS152     | C-302     | 2:00 PM   | Sajid Khan      
----------------------------------------------------------